# Comparación perceptrón

¿Recuerdas el ejercicio del parcial? Vamos a repetirlo pero utilizando *k*-NN y perceptrón para comparar su desempeño. Ten en cuenta las recomendaciones realizadas en el parcial (principalmente la de el pre-procesamiento de las muestras del archivo $\verb|Test-txt|$)

El sistema experto que generes debe cumploir con las siguientes condiciones:

1. Se debe hacer una partición de entrenamiento y validación para probar el sistema desarrollado.
2. Tener una etapa de preprocesamiento de las observaciones.
3. ***Comparar el desempeño del perceptrón con el desempeño de *k*-NN (utilizando la librería de python)***
4. Después de tener el sistema entrenado y validado se debe dar la información  de calidad para las 19 muestras de café de las que no se tiene conocimiento.
5. Cualquier tipo de información adicional que crea requerir la debe suponer. Cada supuesto debe estar perfectamente justificado de forma explícita en formato de comentarios dentro del código


In [1]:
import numpy as np
# from sklearn.model_selection import train_test_split
import math
from numpy import genfromtxt
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn import neighbors, datasets
import neurolab as nl

#Se cargan los datos de los archivos de texto a cada una de sus variables:
# cafe.txt en cafe, expertos.txt en expertos y test.txt en test

cafe = genfromtxt('cafe.txt', delimiter=',')
print(cafe.shape)

expertos = genfromtxt('expertos.txt', delimiter=',') 
print(expertos.shape)

test = genfromtxt('test.txt', delimiter=',')
print(test.shape)

(100, 5)
(100, 4)
(20, 5)


In [2]:
#Definiciones de funciones

#Hacemos la función para la remoción de la media, nuestra idea es que la desviación estándar sea 1 y la media sea 0
def Remocion(X): #definimos la función
    X = X - X.mean(axis=0) # a los datos les restamos el valor de la media
    X = X/X.std(axis=0) #al resultado lo dividimos por la desviación estándar
    return X #retornamos los datos normalizamos

#Método para elegir y retornar una etiqueta para un café usando la regla mayoritaria en las calificaciones de los expertos
def regla_mayoritaria_vector(vector):
    #etiqueta es un vector de dos posiciones, donde el primer valor es la etiqueta que más se repite en el vector
    # y el segundo valor es cuantas veces está en el vector
    etiqueta = np.array([0,0]) 
    for i in range(len(np.unique(vector))): 
        #contador es el numero de veces que una etiqueta se encuentra en el vector
        contador = np.shape(np.where(vector == np.unique(vector)[i]))[1]
        if etiqueta[1] < contador: #Si el contador es mayor a la etiqueta anterior o por defecto, se guardan los datos
            etiqueta[1] = contador #Numero de veces que se encontró
            etiqueta[0] = np.unique(vector)[i] #Cual etiqueta es
    #En caso de encontrar un número igual de ocurrencias para dos etiquetas, se tomará la primera etiqueta encontrada
    return etiqueta

#Método para elegir y retornar la etiqueta para todos las muestras de café usando la regla mayoritaria
def regla_mayoritaria(matriz):
    filas = np.shape(matriz)[0]
    etiquetas = np.array([]) #Se declara un array para agrupar todas las etiquetas
    for i in range(filas):
        etiqueta = regla_mayoritaria_vector(matriz[i]) #Obtengo la etiqueta para cada fila (muestra de café)
        etiquetas = np.append(etiquetas,etiqueta[0]) #La añado al array temporal
    return etiquetas
   

In [3]:
#Punto 2
#Le aplico remoción de la media a todos las observaciones de las muestras de café y las muestras en test.txt
X = Remocion(cafe)
test_remocion = Remocion(test)

#Usando la regla mayoritaria hallo la etiqueta para cada muestra de café
#y = regla_mayoritaria(expertos)
y = expertos
# print(y)

In [4]:
#Punto 1
#Divido la matriz de observaciones en 70% para entrenamiento y 30% para validación
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)
print(X_train.shape, '\n', X_test.shape)
print(y_train.shape, '\n', y_test.shape)

(70, 5) 
 (30, 5)
(70, 4) 
 (30, 4)


In [5]:
#Punto 3 
#Clasificador KNN usando librería de python
y_train_KNN = regla_mayoritaria(y_train)
y_test_KNN = regla_mayoritaria(y_test)
NN = 9 #definimos el número de vecinos (en lo posible elegir números impares)
clasificador_KNN = neighbors.KNeighborsClassifier(NN, weights = 'distance') #instanciamos el clasificador
clasificador_KNN.fit(X_train,y_train_KNN) #entrenamos el clasificador

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='distance')

In [6]:
print(y_train[:,0].shape)
y_train_stack = np.vstack(y_train[:,0])
print(y_train_stack.shape)

(70,)
(70, 1)


In [7]:
#Perceptrón
#y_train_stack = np.vstack(y_train)
#y_test_stack = np.vstack(y_test)

def minimo_y_maximo(train,test):
    minimo = train.min()
    if minimo > test.min():
        minimo = test.min()
    maximo = train.max()
    if maximo < test.max():
        maximo = test.max()
    print(minimo,maximo)
    return minimo, maximo

#definimos los valores máximos y mínimos que puede tomar cada dimensión
# dim1_min, dim1_max, dim2_min, dim2_max = 0,1,0,1
#dim1_min, dim1_max = minimo_y_maximo(X_train,X_test)
#dim2_min, dim2_max = minimo_y_maximo(y_train_stack,y_test_stack)

dim1_min, dim1_max = minimo_y_maximo(X_train[0],X_test[0])
dim2_min, dim2_max = minimo_y_maximo(X_train[1],X_test[1])
dim3_min, dim3_max = minimo_y_maximo(X_train[2],X_test[2])
dim4_min, dim4_max = minimo_y_maximo(X_train[3],X_test[3])
dim5_min, dim5_max = minimo_y_maximo(X_train[4],X_test[4])

#Dado que los datos están seprados en dos clases, solo necesitamos un bit
#para representar la salida. Por tanto la capa de salida solo tendrá una neurona

#Número de neuronas en la capa de salida
num_output = y_train.shape[1]
#tenemos un conjunto de datos de dos dimensiones, definimos un perceptron
# con dos neuronas a la entrada y le asignamos una a cada dimensión
dim1 = [dim1_min, dim1_max]
dim2 = [dim2_min, dim2_max]
dim3 = [dim3_min, dim3_max]
dim4 = [dim4_min, dim4_max]
dim5 = [dim5_min, dim5_max]

#perceptron = nl.net.newp([dim1,dim2], num_output)
perceptron = nl.net.newp([dim1,dim2,dim3,dim4,dim5], num_output)
#entrenamos el perceptron con el conjunto de entrenamiento 
# lr es la tasa de aprendizaje, e indica que tan rápico o que tan lento se actualizan los pasos
#epochs indica el número de pasadas por todo el conjunto de entrenamiento
progreso_error = perceptron.train(X_train, y_train, epochs=100, show=20, lr=0.03)
perceptron.layers[0].np['w'] #con esta instrucción conocemos los pesos para la capa de entrada

-1.6552544681886114 1.506188702633783
-1.4206026640535003 1.0929682080840482
-1.179744511939493 1.8173073531860975
-1.4206026640535003 0.8863579608091808
-1.224036862008721 2.365010055045843
Epoch: 20; Error: 15.0;
Epoch: 40; Error: 16.5;
Epoch: 60; Error: 17.5;
Epoch: 80; Error: 11.0;
Epoch: 100; Error: 14.5;
The maximum number of train epochs is reached


array([[-0.02251978,  0.04375428, -0.09637135,  0.09898697,  0.00766844],
       [ 0.23391514,  0.06528416,  0.02292914, -0.15117672, -0.01775485],
       [-0.12228483,  0.11633082, -0.01982595,  0.07716893,  0.06459448],
       [-0.02929993,  0.05972806, -0.02551513, -0.10016465,  0.00635582]])

In [8]:
#Punto 3
#Validamos el acierto de clasificación del clasificador KNN
y_pred_KNN = clasificador_KNN.predict(X_test) #realizamos la predicción de la etiqueta para el conjunto de validación
acc_KNN = 100.0*(y_test_KNN == y_pred_KNN).sum()/X_test.shape[0] #calculamos el acierto de clasificación
print('El acierto de clasificación del clasificador KNN (python) es del ',acc_KNN , '%') #imprimimos el acierto de clasificacion
print('Las muestras mal clasificadas del clasificador KNN fueron %d' % (y_test_KNN != y_pred_KNN).sum())

#vamos a validar con el mismo conjunto de entrenamiento. Recuerda que la validación se hace normalmente con el conjunto de test
#Validamos el acierto de clasficiación del perceptrón    
y_pred_perceptron = perceptron.sim(X_test)
print('El acierto de clasificación es: ', (y_test==y_pred_perceptron).sum()/(np.shape(y_test)[0]*np.shape(y_test)[1])*100, '%')
print((y_test == y_pred_perceptron).sum(), y_pred_perceptron.shape, y_test.shape, len(y_test))
print(y_pred_perceptron, y_test)

El acierto de clasificación del clasificador KNN (python) es del  90.0 %
Las muestras mal clasificadas del clasificador KNN fueron 3
El acierto de clasificación es:  83.33333333333334 %
100 (30, 4) (30, 4) 30
[[0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 1. 0.]
 [1. 0. 1. 0.]
 [1. 0. 1. 0.]
 [1. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 1. 0. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 1. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 1. 1. 1.]
 [0. 1. 1. 0.]
 [0. 0. 1. 0.]] [[0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [1. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 0. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0.

In [9]:
test_remocion = Remocion(test)

y_pred_test_KNN = clasificador_KNN.predict(test_remocion) 
print("Las predicciones para cada uno de las muestras en test.txt usando el clasificador KNN(python) es: \n")
print(y_pred_test_KNN)

y_pred_perceptron_test = perceptron.sim(test_remocion)
print(y_pred_perceptron_test)

Las predicciones para cada uno de las muestras en test.txt usando el clasificador KNN(python) es: 

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]
 [1. 1. 1. 1.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 1. 0.]
 [1. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 1. 1.]
 [0. 1. 0. 0.]
 [0. 1. 1. 0.]]
